In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import plotly
import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

plotly.tools.set_credentials_file(username='SamirMamedau', api_key='mpEvqAnZPqtls0Szi2Ga')
init_notebook_mode(connected=True)

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Load Data

In [2]:
data = pd.read_pickle('mainData2020.csv')

eu = list(range(0, 80))
na = list(range(80, 160))
kr = list(range(160, 273))
regions = [eu, na, kr, 'FNC', 'VIT', 'G2', 'OG', 'TL', 'C9', 'GRF', 'SKT', 'RGE', '100', 'JAG']
teamsAll = np.unique(data.loc['Static', 'playerTeam'][[0,1]].values.ravel())

In [3]:
teamsAll

array(['100', 'AF', 'APK', 'C9', 'CLG', 'DIG', 'DRX', 'DWG', 'EG', 'FLY',
       'FNC', 'G2', 'GEN', 'GG', 'GRF', 'HLE', 'IMT', 'KT', 'MAD', 'MSF',
       'OG', 'RGE', 'S04', 'SB', 'SK', 'T1', 'TL', 'TSM', 'VIT', 'XL'],
      dtype=object)

## Lanes Data

In [4]:
# GoldTotal, GoldCurrent, Minions, JungleMinions, Level, XP
def lanesDynamic(stat, time, region):
    
    res = []
    if region not in teamsAll:
        for i in range(2, 7):
            res.append(data.loc['Dynamic', stat].loc[region][[i, i+5]].xs(time, level = 1).mean().mean())
            
    else:
        for i in range(2, 7):
            dfMain = data.loc['Dynamic', stat][[i, i+5]].xs(time, level = 1)
            dfTeam = data.loc['Static', 'playerTeam'][[0,1]] == region
            dfTeam.columns = [i, i+5]
            res.append(dfMain[dfTeam].mean().mean())
            
    return pd.DataFrame(res).T

In [5]:
# Kills, Deaths, Assists, GoldFinal, Minions, JungleMinions, EnemyJungleMinions, (Total)DamageDealt/Taken, TotalHeal,
# PinkWards, WardsPlaced, WardsKilled
def lanesStatic(stat, region):
    
    res = []
    if region not in teamsAll:
        for i in range(2, 7):
            res.append(data.loc['Static', stat].loc[region][[i, i+5]].mean().mean())
            
    else: 
        for i in range(2, 7):
            dfMain = data.loc['Static', stat][[i, i+5]]
            dfTeam = data.loc['Static', 'playerTeam'][[0,1]] == region
            dfTeam.columns = [i, i+5]
            res.append(dfMain[dfTeam].mean().mean())
        
    return pd.DataFrame(res).T

In [6]:
# GoldTotal, GoldCurrent, Minions, JungleMinions, Level, XP
def statOverTime(stat, team):
    
    yMean = [round(data.loc['Dynamic', stat][[0,1]].xs(time, level = 1).mean().mean()) for time in range(30)]
    y = [round(lanesDynamic(stat, time, team).loc[0].sum()) for time in range(30)]
                                   
    return [a - b for a,b in zip(y, yMean)]

In [7]:
def goldGrowth(team):
    
    y = [lanesDynamic('goldTotal', time, team).loc[0].sum() for time in range(30)]
    y = [round(res[i+1] - res[i], 0) for i,x in enumerate(res[:-1])]
    
    yMean = [round(data.loc['Dynamic', 'goldTotal'][[0,1]].xs(time, level = 1).mean().mean(), 0) for time in range(30)]
    yMean = [round(yMean[i+1] - yMean[i], 0) for i,x in enumerate(yMean[:-1])]
    
    return [a - b for a,b in zip(y, yMean)]

## Dragon

In [8]:
def DrakeOT(team, time):

    res = data.loc['Dynamic', 'dragonKiller'][[0,1]]
    Time0 = data.loc['Dynamic', 'dragonTime'][[0,1]] > 0
    Time1 = data.loc['Dynamic', 'dragonTime'][[0,1]] < time
    team = data.loc['Static', 'playerTeam'][[0,1]] == team
    team.index = team.index.droplevel(level=1)
    us = data.loc['Dynamic', 'dragonKiller'][[0,1]]==True

    return np.round(res[Time0][Time1][us][team].sum().sum() / team.sum().sum(), 2)

In [9]:
DragonTeams = ['FNC', 'G2', 'VIT', 'SK', 'S04', 'OG', 'T1', 'DRX', 'GEN']
x = list(range(0, 31))

In [10]:
iplot({
    'data': [Scatter(x = x,
                     y = [DrakeOT(team, x) for x in x],
                     name = team) for team in DragonTeams],
    
    'layout': Layout(width = 1000, height = 600,
                     title = 'Dragons over Time',
                     xaxis = dict(title = 'Time'),
                     yaxis = dict(title = 'Dragons'))
})

## Vision

In [11]:
# YELLOW_TRINKET, SIGHT_WARD, BLUE_TRINKET, CONTROL_WARD
def Wards(wardType, team, time1):
    
    dfType = data.loc['Dynamic', 'wardPlacedType'][[0,1]] == wardType
    dfCreator = data.loc['Dynamic', 'wardCreator'][[0,1]] == True
    dfTime1 = data.loc['Dynamic', 'wardPlacedTime'][[0,1]] < time1
    dfTeam = data.loc['Static', 'playerTeam'][[0,1]] == team
    dfTeam.columns = [0,1]
    dfTeam.index = dfTeam.index.droplevel(level=1)
            
    return round(dfType[dfCreator][dfTime1][dfTeam].sum().sum() / dfTeam.sum().sum(), 1)

In [12]:
def wardsOverTime(wardType, team):
    
    yMean = []
    dfType = data.loc['Dynamic', 'wardPlacedType'] == wardType
    for time in range(30):
        dfTime = data.loc['Dynamic', 'wardPlacedTime'] < time
        yMean.append(round(dfType[dfTime].sum()[0] / (dfTime.index.levels[0][-1] * 2), 1))
    
    y = [Wards(wardType, team, time) for time in range(30)]
                                   
    return [a - b for a,b in zip(y, yMean)]

In [13]:
iplot({
    'data': [Scatter(x = list(range(30)),
                     y = np.array(wardsOverTime('YELLOW_TRINKET', team)) + np.array(wardsOverTime('SIGHT_WARD', team)),
                     name = team)
             for team in ['G2', 'FNC', 'VIT', 'C9', 'T1', 'GEN', 'SK']]
})

## Plot

In [14]:
plotTeams = ['G2', 'FNC', 'OG', 'MAD', 'MSF', 'RGE', 'XL', 'S04', 'SK', 'VIT']

dataGold = [Scatter(x = list(range(31)),
                    y = statOverTime('goldTotal', team), 
                    mode = 'lines',
                    name = team) 
                    for team in plotTeams]

dataMinions = [Scatter(x = list(range(31)),
                       y = statOverTime('minions', team), 
                       mode = 'lines',
                       name = team) 
                       for team in plotTeams]

dataXP = [Scatter(x = list(range(31)),
                  y = statOverTime('xp', team), 
                  mode = 'lines',
                  name = team) 
                  for team in plotTeams]

dataDrakes =  [Scatter(x = list(range(31)),
                       y = [DrakeOT(team, time) for time in x],
                       mode = 'lines',
                       name = team) 
                       for team in plotTeams]

dataWards = [Scatter(x = list(range(31)),
                     y = np.array(wardsOverTime('YELLOW_TRINKET', team)) + np.array(wardsOverTime('SIGHT_WARD', team)),
                     mode = 'lines',
                     name = team)
                     for team in plotTeams]

In [15]:
buttonsGold = list([dict(label = '   Gold    ', method = 'update',
                  args = [{'visible': [True] * 10 + [False] * 40}, 
                          {'title': 'Gold Difference'}])])

buttonsMinions = list([dict(label = 'Minions ', method = 'update',
                  args = [{'visible': [False] * 10 + [True] * 10 + [False] * 30}, 
                          {'title': 'Minions Difference'}])])

buttonsXP = list([dict(label = '     XP     ', method = 'update',
                  args = [{'visible': [False] * 20 + [True] * 10 + [False] * 20}, 
                          {'title': 'XP Difference'}])])

buttonsDrakes = list([dict(label = ' Drakes ', method = 'update',
                  args = [{'visible': [False] * 30 + [True] * 10 + [False] * 10}, 
                          {'title': 'Drakes Taken'}])])

buttonsWards = list([dict(label = '  Wards  ', method = 'update',
                  args = [{'visible': [False] * 40 + [True] * 10}, 
                          {'title': 'Wards Placed Difference'}])])

In [16]:
iplot({
    'data': dataGold + dataMinions + dataXP + dataDrakes + dataWards,
    
    'layout': Layout(width = 900, height = 600,
                     xaxis = dict(title = 'Time'),
                     yaxis = dict(title = 'Stat'),
                     updatemenus = list([dict(type = "buttons", active = -1, buttons = buttonsGold, x = -0.1, y = 0.95),
                                         dict(type = "buttons", active = -1, buttons = buttonsMinions, x = -0.1, y = 0.87),
                                         dict(type = "buttons", active = -1, buttons = buttonsXP, x = -0.1, y = 0.79),
                                         dict(type = "buttons", active = -1, buttons = buttonsDrakes, x = -0.1, y = 0.71),
                                         dict(type = "buttons", active = -1, buttons = buttonsWards, x = -0.1, y = 0.63)]))
})

## Playground

In [17]:
def heraldSecure(team):
    
    us = data.loc['Static', 'playerTeam'].xs(0, level = 1)[[0,1]] == team
    kill_1 = data.loc['Dynamic', 'heraldKiller'].xs(0, level = 1)[[0,1]] == True
    dontKill_1 = data.loc['Dynamic', 'heraldKiller'].xs(0, level = 1)[[0,1]] == False
    kill_2 = data.loc['Dynamic', 'heraldKiller'].xs(1, level = 1)[[0,1]] == True
    dontKill_2 = data.loc['Dynamic', 'heraldKiller'].xs(1, level = 1)[[0,1]] == False
    
    return round(kill_1[us].sum().sum() / us.sum().sum(), 2) * 100, round(kill_2[us].sum().sum() / us.sum().sum(), 2) * 100

In [18]:
def heraldTime(team):
    
    us = data.loc['Static', 'playerTeam'].xs(0, level = 1)[[0,1]] == team
    time_1 = data.loc['Dynamic', 'heraldTime'].xs(0, level = 1)[[0,1]]
    time_2 = data.loc['Dynamic', 'heraldTime'].xs(1, level = 1)[[0,1]]
    kill_1 = data.loc['Dynamic', 'heraldKiller'].xs(0, level = 1)[[0,1]] == True
    kill_2 = data.loc['Dynamic', 'heraldKiller'].xs(1, level = 1)[[0,1]] == True
    
    res_1 = set(time_1[kill_1][us].values.ravel())
    res_1 = [x for x in res_1 if x == x]
    res_2 = set(time_2[kill_2][us].values.ravel())
    res_2 = [x for x in res_2 if x == x]
    
    return round(mean(res_1), 1), round(mean(res_2), 1)

In [19]:
def heraldGold(team):
    
    us = data.loc['Static', 'playerTeam'].xs(0, level = 1)[[0,1]] == team
    time_1 = data.loc['Dynamic', 'heraldTime'].xs(0, level = 1)[[0,1]]
    time_2 = data.loc['Dynamic', 'heraldTime'].xs(1, level = 1)[[0,1]]
    kill_1 = data.loc['Dynamic', 'heraldKiller'].xs(0, level = 1)[[0,1]] == True
    kill_2 = data.loc['Dynamic', 'heraldKiller'].xs(1, level = 1)[[0,1]] == True
    
    res1 = []
    for i in time_1[kill_1][us].index:
        try:
            t = np.floor(time_1[kill_1][us].loc[i][0])
            GD = data.loc['Dynamic', 'goldTotal', i][0][t] - data.loc['Dynamic', 'goldTotal', i][1][t]
            res1.append(GD)
        except:
            pass
        try:
            t = np.floor(time_1[kill_1][us].loc[i][1])
            GD = data.loc['Dynamic', 'goldTotal', i][1][t] - data.loc['Dynamic', 'goldTotal', i][0][t]
            res1.append(GD)
        except:
            pass
        
    res2 = []
    for i in time_2[kill_2][us].index:
        try:
            t = np.floor(time_2[kill_2][us].loc[i][0])
            GD = data.loc['Dynamic', 'goldTotal', i][0][t] - data.loc['Dynamic', 'goldTotal', i][1][t]
            res2.append(GD)
        except:
            pass
        try:
            t = np.floor(time_2[kill_2][us].loc[i][1])
            GD = data.loc['Dynamic', 'goldTotal', i][1][t] - data.loc['Dynamic', 'goldTotal', i][0][t]
            res2.append(GD)
        except:
            pass
    
    return round(mean(res1), 1), round(mean(res2), 1)

In [20]:
def heraldSummon(team):
    
    us = data.loc['Static', 'playerTeam'].xs(0, level = 1)[[0,1]] == team
    time_1 = data.loc['Dynamic', 'heraldSummon'].xs(0, level = 1)[[0,1]]
    time_2 = data.loc['Dynamic', 'heraldSummon'].xs(1, level = 1)[[0,1]]
    kill_1 = data.loc['Dynamic', 'heraldKiller'].xs(0, level = 1)[[0,1]] == True
    kill_2 = data.loc['Dynamic', 'heraldKiller'].xs(1, level = 1)[[0,1]] == True
    
    res_1 = set(time_1[kill_1][us].values.ravel())
    res_1 = [x for x in res_1 if x == x]
    res_2 = set(time_2[kill_2][us].values.ravel())
    res_2 = [x for x in res_2 if x == x]
    
    return round(mean(res_1), 1), round(mean(res_2), 1)

In [21]:
def heraldPP(team):
    
    us = data.loc['Static', 'playerTeam'].xs(0, level = 1)[[0,1]] == team
    time_1 = data.loc['Dynamic', 'heraldSummon'].xs(0, level = 1)[[0,1]]
    time_2 = data.loc['Dynamic', 'heraldSummon'].xs(1, level = 1)[[0,1]]
    kill_1 = data.loc['Dynamic', 'heraldKiller'].xs(0, level = 1)[[0,1]] == True
    kill_2 = data.loc['Dynamic', 'heraldKiller'].xs(1, level = 1)[[0,1]] == True
    
    res1 = []
    for i in time_1[kill_1][us].index:
        try:
            t = np.ceil(time_1[kill_1][us].loc[i][0])
            GD1 = data.loc['Dynamic', 'goldTotal', i][0][t] - data.loc['Dynamic', 'goldTotal', i][1][t]
            GD2 = data.loc['Dynamic', 'goldTotal', i][0][t+2] - data.loc['Dynamic', 'goldTotal', i][1][t+2]
            res1.append((GD2 - GD1)/2)
        except:
            pass
        try:
            t = np.ceil(time_1[kill_1][us].loc[i][1])
            GD1 = data.loc['Dynamic', 'goldTotal', i][1][t] - data.loc['Dynamic', 'goldTotal', i][0][t]
            GD2 = data.loc['Dynamic', 'goldTotal', i][1][t+2] - data.loc['Dynamic', 'goldTotal', i][0][t+2]
            res1.append((GD2 - GD1)/2)
        except:
            pass
        
    res2 = []
    for i in time_2[kill_2][us].index:
        try:
            t = np.ceil(time_2[kill_2][us].loc[i][0])
            GD1 = data.loc['Dynamic', 'goldTotal', i][0][t] - data.loc['Dynamic', 'goldTotal', i][1][t]
            GD2 = data.loc['Dynamic', 'goldTotal', i][0][t+2] - data.loc['Dynamic', 'goldTotal', i][1][t+2]
            res2.append((GD2 - GD1)/2)
        except:
            pass
        try:
            t = np.ceil(time_2[kill_2][us].loc[i][1])
            GD1 = data.loc['Dynamic', 'goldTotal', i][1][t] - data.loc['Dynamic', 'goldTotal', i][0][t]
            GD2 = data.loc['Dynamic', 'goldTotal', i][1][t+2] - data.loc['Dynamic', 'goldTotal', i][0][t+2]
            res2.append((GD2 - GD1)/2)
        except:
            pass
    
    return round(mean(res1), 1), round(mean(res2), 1)

In [22]:
teamsEU = ['G2', 'FNC', 'OG', 'MAD', 'MSF', 'RGE', 'XL', 'S04', 'SK', 'VIT']
teamsNA = ['C9', 'EG', 'FLY', 'TSM', '100', 'IMT', 'TL', 'DIG', 'GG', 'CLG']
teamsKR = ['GEN', 'T1', 'DRX', 'AF', 'KT', 'DWG', 'HLE', 'SB', 'APK', 'GRF']
pd.DataFrame([heraldSummon(team) for team in teamsKR])

,0,1
0,11.4,17.8
1,12.6,17.5
2,12.4,18.0
3,12.7,18.3
4,11.7,17.4
5,11.8,19.0
6,11.6,17.9
7,12.1,18.7
8,11.9,18.2
9,13.6,19.3
